In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import normalize
from sklearn.metrics import normalized_mutual_info_score
import matplotlib.pyplot as plt

from SSC import sparseSubspaceClustering
import preprocessing
import simulations as sims
import RelatednessTesting as rt
import SPADAutil as util

In [2]:
seger = pd.read_csv("/Users/fcseidl/Documents/SPADA/RNAseq/human_pancreas/Segerstolpe/data.tsv", delimiter='\t', dtype=str)

In [3]:
seger_lbl = pd.read_csv("/Users/fcseidl/Documents/SPADA/RNAseq/human_pancreas/Segerstolpe/labels.tsv", delimiter='\t', dtype=str)
xin_lbl = pd.read_csv("/Users/fcseidl/Documents/SPADA/RNAseq/human_pancreas/Xin/labels.tsv", delimiter='\t', dtype=str)

In [4]:
xin = pd.read_csv("/Users/fcseidl/Documents/SPADA/RNAseq/human_pancreas/Xin/data.tsv", delimiter='\t', dtype=str)

In [5]:
seger_lbl

,Source Name,Comment[ENA_SAMPLE],Comment[BioSD_SAMPLE],Characteristics[organism],Characteristics[organism part],Characteristics[individual],Characteristics[single cell well quality],Characteristics[cell type],Characteristics[disease],Characteristics[sex],...,Comment[ENA_EXPERIMENT],Scan Name,Comment[SUBMITTED_FILE_NAME],Comment[ENA_RUN],Comment[FASTQ_URI],Protocol REF.3,Derived Array Data File,Comment [Derived ArrayExpress FTP file],Factor Value[disease],Factor Value[cell type]
0,AZ_A1,ERS1348470,SAMEA4437021,Homo sapiens,pancreas,AZ,low quality cell,not applicable,normal,male,...,ERX1700346,AZ_A1.fastq.gz,AZ_A1.fastq.gz,ERR1630013,ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR163/003/...,P-MTAB-51966,pancreas_refseq_rpkms_counts_3514sc.txt,ftp://ftp.ebi.ac.uk/pub/databases/microarray/d...,normal,not applicable
1,AZ_A10,ERS1348471,SAMEA4437022,Homo sapiens,pancreas,AZ,OK,delta cell,normal,male,...,ERX1700347,AZ_A10.fastq.gz,AZ_A10.fastq.gz,ERR1630014,ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR163/004/...,P-MTAB-51966,pancreas_refseq_rpkms_counts_3514sc.txt,ftp://ftp.ebi.ac.uk/pub/databases/microarray/d...,normal,delta cell
2,AZ_A11,ERS1348472,SAMEA4437023,Homo sapiens,pancreas,AZ,OK,alpha cell,normal,male,...,ERX1700348,AZ_A11.fastq.gz,AZ_A11.fastq.gz,ERR1630015,ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR163/005/...,P-MTAB-51966,pancreas_refseq_rpkms_counts_3514sc.txt,ftp://ftp.ebi.ac.uk/pub/databases/microarray/d...,normal,alpha cell
3,AZ_A12,ERS1348473,SAMEA4437024,Homo sapiens,pancreas,AZ,OK,delta cell,normal,male,...,ERX1700349,AZ_A12.fastq.gz,AZ_A12.fastq.gz,ERR1630016,ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR163/006/...,P-MTAB-51966,pancreas_refseq_rpkms_counts_3514sc.txt,ftp://ftp.ebi.ac.uk/pub/databases/microarray/d...,normal,delta cell
4,AZ_A2,ERS1348474,SAMEA4437025,Homo sapiens,pancreas,AZ,OK,gamma cell,normal,male,...,ERX1700350,AZ_A2.fastq.gz,AZ_A2.fastq.gz,ERR1630017,ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR163/007/...,P-MTAB-51966,pancreas_refseq_rpkms_counts_3514sc.txt,ftp://ftp.ebi.ac.uk/pub/databases/microarray/d...,normal,gamma cell
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3509,HP1526901T2D_P5,ERS1351979,SAMEA4440530,Homo sapiens,pancreas,HP1526901T2D,low quality cell,not applicable,type II diabetes mellitus,female,...,ERX1703855,HP1526901T2D_P5.fastq.gz,HP1526901T2D_P5.fastq.gz,ERR1633522,ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR163/002/...,P-MTAB-51966,pancreas_refseq_rpkms_counts_3514sc.txt,ftp://ftp.ebi.ac.uk/pub/databases/microarray/d...,type II diabetes mellitus,not applicable
3510,HP1526901T2D_P6,ERS1351980,SAMEA4440531,Homo sapiens,pancreas,HP1526901T2D,low quality cell,not applicable,type II diabetes mellitus,female,...,ERX1703856,HP1526901T2D_P6.fastq.gz,HP1526901T2D_P6.fastq.gz,ERR1633523,ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR163/003/...,P-MTAB-51966,pancreas_refseq_rpkms_counts_3514sc.txt,ftp://ftp.ebi.ac.uk/pub/databases/microarray/d...,type II diabetes mellitus,not applicable
3511,HP1526901T2D_P7,ERS1351981,SAMEA4440532,Homo sapiens,pancreas,HP1526901T2D,OK,beta cell,type II diabetes mellitus,female,...,ERX1703857,HP1526901T2D_P7.fastq.gz,HP1526901T2D_P7.fastq.gz,ERR1633524,ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR163/004/...,P-MTAB-51966,pancreas_refseq_rpkms_counts_3514sc.txt,ftp://ftp.ebi.ac.uk/pub/databases/microarray/d...,type II diabetes mellitus,beta cell
3512,HP1526901T2D_P8,ERS1351982,SAMEA4440533,Homo sapiens,pancreas,HP1526901T2D,low quality cell,not applicable,type II diabetes mellitus,female,...,ERX1703858,HP1526901T2D_P8.fastq.gz,HP1526901T2D_P8.fastq.gz,ERR1633525,ftp://ftp.sra.ebi.ac.uk/vol1/fastq/ERR163/005/...,P-MTAB-51966,pancreas_refseq_rpkms_counts_3514sc.txt,ftp://ftp.ebi.ac.uk/pub/databases/microarray/d...,type II diabetes mellitus,not applicable


In [6]:
xin_lbl

,Sample.name,title,source.name,organism,tissue,molecule,processed.data.file,raw.file,donor.id,condition,age,ethnicity,gender,cell.type
0,Sample 1,Islet cell,Pancreatic islets,Homo sapiens,Pancreatic islets,mRNA,expr_rpkm.txt,Sample_1.fq.gz,Non T2D 1,Healthy,23,AA,M,beta
1,Sample 2,Islet cell,Pancreatic islets,Homo sapiens,Pancreatic islets,mRNA,expr_rpkm.txt,Sample_2.fq.gz,Non T2D 1,Healthy,23,AA,M,beta
2,Sample 3,Islet cell,Pancreatic islets,Homo sapiens,Pancreatic islets,mRNA,expr_rpkm.txt,Sample_3.fq.gz,Non T2D 1,Healthy,23,AA,M,beta
3,Sample 4,Islet cell,Pancreatic islets,Homo sapiens,Pancreatic islets,mRNA,expr_rpkm.txt,Sample_4.fq.gz,Non T2D 1,Healthy,23,AA,M,beta
4,Sample 5,Islet cell,Pancreatic islets,Homo sapiens,Pancreatic islets,mRNA,expr_rpkm.txt,Sample_5.fq.gz,Non T2D 1,Healthy,23,AA,M,beta
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1595,Sample 1596,Islet cell,Pancreatic islets,Homo sapiens,Pancreatic islets,mRNA,expr_rpkm.txt,Sample_1596.fq.gz,T2D 1,T2D,57,AA,M,PP.contaminated
1596,Sample 1597,Islet cell,Pancreatic islets,Homo sapiens,Pancreatic islets,mRNA,expr_rpkm.txt,Sample_1597.fq.gz,T2D 2,T2D,37,C,F,PP
1597,Sample 1598,Islet cell,Pancreatic islets,Homo sapiens,Pancreatic islets,mRNA,expr_rpkm.txt,Sample_1598.fq.gz,T2D 5,T2D,42,C,M,PP
1598,Sample 1599,Islet cell,Pancreatic islets,Homo sapiens,Pancreatic islets,mRNA,expr_rpkm.txt,Sample_1599.fq.gz,T2D 5,T2D,42,C,M,PP.contaminated


In [7]:
xin

,symbol,Sample_1,Sample_2,Sample_3,Sample_4,Sample_5,Sample_6,Sample_7,Sample_8,Sample_9,...,Sample_1591,Sample_1592,Sample_1593,Sample_1594,Sample_1595,Sample_1596,Sample_1597,Sample_1598,Sample_1599,Sample_1600
0,A1BG,47.3396,24.0458,2.2743,0,0,0,0,0,0,...,0,0.3304,2.4857,0,8.1498,0,0.5372,31.1225,0,0.5788
1,A2M,0,0,0,0,0,0,0,0.254,0,...,0,0,0,0,0,0,0,0,47.4664,0
2,A2MP1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,NAT1,0,0,0,0,130.9796,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,NAT2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39846,LOC101929771,0,0,0,0,0,0,0,0,0.18,...,0,0,0,0,0,0.0721,0,0,0,0
39847,LOC101930100,0,0,3.13,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.4338
39848,LOC102723951,0.2423,0,0,0,0,0,0,0.1175,0,...,0,0,0,0,0,0,0.099,0,0,0
39849,LOC102724004,0.2423,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
seger

,gene,samples,HP1502401_N13,HP1502401_D14,HP1502401_F14,HP1502401_J13,HP1502401_B13,HP1502401_H13,HP1502401_J14,HP1502401_B14,...,HP1525301T2D_O10,HP1526901T2D_H2,HP1526901T2D_I16,HP1526901T2D_F7,HP1526901T2D_I23,HP1525301T2D_K3,HP1525301T2D_J10,HP1526901T2D_N8,HP1526901T2D_O11,HP1526901T2D_A8
0,SGIP1,NM_032291,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,4.86178584602,0.0,3.10298699593,0.0,0.0,0.0,0.0
1,AZIN2,NM_052998+NM_001293562,0.0,0.0,0.0,0.0,0.0,41.5192874825,0.0,0.0,...,0.0,0.0,0.0,24.7922028911,0.0,0.0,0.0,0.0,0.0,14.6712184029
2,CLIC4,NM_013943,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,14.12431332,9.32527756841,44.743016863,13.2348092565,0.0,0.0,0.0,0.0,0.0,1.74042870544
3,AGBL4,NM_032785,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,7.47693331678,0.0,0.0,0.0
4,NECAP2,NM_001145277+NM_001145278+NM_018090,0.0,0.0,0.0,0.0,0.0,13.9927247899,0.0,18.9243473685,...,0.0,0.0,0.0,22.7870421979,0.0,1.74981030261,104.741197234,17.7444760065,0.0,22.2500553962
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26266,ERCC_0.22888184:mix1_0.45776367:mix2,ERCC-00081,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26267,ERCC_0.11444092:mix1_0.22888184:mix2,ERCC-00086,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26268,ERCC_0.05722046:mix1_0.11444092:mix2,ERCC-00061,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26269,ERCC_0.01430512:mix1_0.02861023:mix2,ERCC-00048,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
